In [1]:
from keras.models import load_model
from numpy import load
from numpy import vstack
from numpy import expand_dims
from matplotlib import pyplot
from numpy.random import randint
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
import os
import shutil
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from PIL import Image


In [2]:

# load and prepare training images
def load_real_samples(filename):
	# load the dataset
	data = load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	# scale from [0,255] to [-1,1]
	X1 = (X1 - 127.5) / 127.5
	X2 = (X2 - 127.5) / 127.5
	return [X1, X2]

# select a random sample of images from the dataset
def select_sample(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	return X

# plot the image, the translation, and the reconstruction
def show_plot(imagesX, imagesY1, imagesY2):
	images = vstack((imagesX, imagesY1, imagesY2))
	titles = ['Real', 'Generated', 'Reconstructed']
	# scale from [-1,1] to [0,1]
	images = (images + 1) / 2.0
	# plot images row by row
	for i in range(len(images)):
		# define subplot
		pyplot.subplot(1, len(images), 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(images[i])
		# title
		pyplot.title(titles[i])
	pyplot.show()

def create_generated_folder(sunny_path):
	#create a generated folder outside the sunny2normal directory
	os.mkdir('generated')
	#loop through the directories in sunny path and make the same directories in the generated path
	for dir_name in os.listdir(sunny_path):
		os.mkdir('generated/' + dir_name)
		#loop through the images in the directory and copy 10 images to the generated directory
		for i, img_name in enumerate(os.listdir(sunny_path + dir_name)):
			if i < 10:
				shutil.copy(sunny_path + dir_name + '/' + img_name, 'generated/' + dir_name + '/' + img_name)


In [19]:

def create_generated_dataset(generated_path):
	cust = {'InstanceNormalization': InstanceNormalization}
	model_BtoA = load_model('./models_sunny/g_model_BtoA_006000_sunny.h5', custom_objects=cust, compile=False)
	os.mkdir('./PlantVillage-Dataset-processed/translated_dataset_sunny/')
	for dir_name in os.listdir(generated_path):
        #make the same dir name in the sunny path
		os.mkdir('./PlantVillage-Dataset-processed/translated_dataset_sunny/' + dir_name)
		#loop through the images in the directory, generate the image and save it in the generated_dataset directory
		for img_name in os.listdir(generated_path + dir_name):
			# load the image
			pixels = load_img(generated_path + dir_name + '/' + img_name, target_size=(256,256))
            # convert to numpy array
			pixels = img_to_array(pixels)
			pixels = expand_dims(pixels, 0)
            # scale from [0,255] to [-1,1]
			pixels = (pixels - 127.5) / 127.5
            # reshape the image
			# pixels = pixels.reshape((1, pixels.shape[0], pixels.shape[1], pixels.shape[2]))
            # generate image from source
			X = model_BtoA.predict(pixels)
            # scale from [-1,1] to [0,1]
			X = (X + 1) / 2.0
			X =  (X * 255.0).astype('uint8')
			#remove pixelation from the image
            # plot the image
			generated_image =   Image.fromarray(X[0])
			filename = './PlantVillage-Dataset-processed/translated_dataset_sunny/' + dir_name + '/' + img_name
			generated_image.save(filename)
			print('Generated: ', filename)



In [20]:
create_generated_dataset('../PlantVillage-Dataset-processed/sunny/')

ValueError: Unknown layer: 'InstanceNormalization'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [ ]:
InstanceNormalization

keras_contrib.layers.normalization.instancenormalization.InstanceNormalization